In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [3]:
vars_procedimientos = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}

In [4]:
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_7140\2020992259.py:1: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,33,37,41,43,47,51,75,76,77,78,79,80,81,82,83,84,85,86,87,123) have mixed types. Specify dtype option on import or set low_memory=False.
  df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)


In [41]:
COLUMNAS_SERVICIO_TRASLADO = {
    "SERVICIOINGRESO": 0,
    "SERVICIOTRASLADO1": 1,
    "SERVICIOTRASLADO2": 2,
    "SERVICIOTRASLADO3": 3,
    "SERVICIOTRASLADO4": 4,
    "SERVICIOTRASLADO5": 5,
    "SERVICIOTRASLADO6": 6,
    "SERVICIOTRASLADO7": 7,
    "SERVICIOTRASLADO8": 8,
    "SERVICIOTRASLADO9": 9,
    "SERVICIOALTA": 10,
}

COLUMNAS_FECHA_TRASLADO = {
    "FECHA_INGRESO": 0,
    "FECHATRASLADO1": 1,
    "FECHATRASLADO2": 2,
    "FECHATRASLADO3": 3,
    "FECHATRASLADO4": 4,
    "FECHATRASLADO5": 5,
    "FECHATRASLADO6": 6,
    "FECHATRASLADO7": 7,
    "FECHATRASLADO8": 8,
    "FECHATRASLADO9": 9,
    "FECHAALTA": 10,
}


# Filtra la base nacional para dejar el Torax
df_torax = df_procesada.query("COD_HOSPITAL == 112103")
# Permite identificar cada egreso
df_torax = df_torax.reset_index().reset_index(names="id_egreso")

In [48]:
servicios_egreso_long = df_torax.melt(
    id_vars="id_egreso",
    value_vars=COLUMNAS_SERVICIO_TRASLADO,
    var_name="tipo_servicio",
    value_name="servicio",
)

fechas_egreso_long = df_torax.melt(
    id_vars="id_egreso",
    value_vars=COLUMNAS_FECHA_TRASLADO,
    var_name="tipo_fecha",
    value_name="fecha",
)

In [51]:
servicio_y_fechas_egreso_contiguas = pd.merge(
    left=fechas_egreso_long,
    right=servicios_egreso_long,
    how="inner",
    left_index=True,
    right_index=True,
)